*14 Nov 2021, Julian Mak (whatever with copyright, do what you want with this)

### As part of material for OCES 3301 "Data Analysis in Ocean Sciences" delivered at HKUST

For the latest version of the material, go to the public facing [GitHub](https://github.com/julianmak/academic-notes/tree/master/OCES3301_data_analysis_ocean) page.

In [ ]:
# load some default packages
import matplotlib.pyplot as plt
import numpy as np

# pull files from the internet if needed (e.g. temporary session in Colab)
# !wget https://raw.githubusercontent.com/julianmak/OCES3301_data_analysis/main/rick_roll.txt
# !wget https://raw.githubusercontent.com/julianmak/OCES3301_data_analysis/main/elnino34_sst.data

# 02: Python data reading and some data manipulation

While a good portion of this course uses data generated on the fly (e.g. see *assignment 1* and some samples from last session), "real" data is usually obtained from observations / numerical experiments / lab work that may be costly or not worthwhile repeating. Data is usually stored somewhere, and we want to be able to read it in order to do something with it.

Most data we are going to deal with here are going to be **text** format (e.g. with the *txt* or *csv* extension, though sometimes no extension at all), and towards the end of the course we are going to start reading some **binary** format data (in this case in the [netCDF](https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/what-is-netcdf-data.htm) format, which is common in geophysical data). This is of course only representative, and while the reading commands might change, the analysis part should be largely similar.

Before we go on and as promised in the previous session, here is a picture of the cursed frog:

<img src="https://i.imgur.com/5qhT8OK.jpg" width="400" alt='cursed frog'>

--------------------

# a) Reading text-based data

Python natively will read text files. The file I made is in `rick_roll.txt`, which you could try and read it outside of Python (it's just a text file, so something like notepad will do it). To read it in Python, here are the somewhat old-fashioned (but very robust) commands you can do:

In [ ]:
txt_file = open("rick_roll.txt", "r")  # r/w/a = read/write/append, so here is "read only"

print(txt_file)

txt_file.close()  # remember to close the file otherwise weird things might happen...

So notice here none of the contents of the file actually got print to screen, because what is going on here is that a **pointer** (in this case the variable `txt_file`) is made to the file, providing an access point of sorts to do things to the file, until it is closed (through `txt_file.close()`). So no reading was actually done because no reading commands were actually issued, which in this case is done with `txt_file.readlines()`.

In [ ]:
txt_file = open("rick_roll.txt", "r")
txt_src = txt_file.readlines()
txt_file.close()

print(txt_src)

> NOTE: `readlines()` is a routine associated to the `txt_file` object, and `txt_file.readlines()` here is saying use the readlines command on itself. The reason for this kind of arrangement (called **object oriented programming**) is that in principle you pack the data together with the routines, so for example you can avoid calling routines to act on stuff that the routine was never designed for (e.g. the computer doesn't necessarily understand what calculating the variance means for a string?) Python is an **object-oriented** language.

Note that we closed the pointer to the file so references to `txt_file` should no longer work, but the data has been dumped out to `txt_src`, which is now the list of strings we can play with (`\n` means new line).

In [ ]:
for i in range(len(txt_src)):
    print(txt_src[i])

The above has lots of line spaces because of the `\n`, so lets try and do something about that. One way is to remove it.

> NOTE: For code development, the approach I take is that "whatever I try first time is probably going to fail". With that in mind, I always try small things before generalising, because then you are dealing with smaller chunks of data that is more manageable, the routines don't take as long to run, and it is easier to debug (e.g. syntax, indexing, failures in logic).
>
> PHILOSOPHICAL: Part of being an "expert" is knowing the ways things can fail, learn from the mistakes, and fail better next time. Never making mistakes to me is in some sense dangerous because there is a risk then one never gets to learn to pick themselves back up. Making mistakes in a way is a "good" thing, as long as one learns a lesson from it and don't repeatedly make the same mistakes.

In [ ]:
# try it for one before doing it for everybody
print("before removal:")
# repr here is representation, so prints out the raw string
# if you don't do that then the simple print just gives you a new line
print(repr(txt_src[0]))
print("after removal using 'replace':")
print(repr(txt_src[0].replace("\n", "")))  # replace with empty string (could use strip too, see below)

In [ ]:
# generalise the above by wrapping it in a for loop
print("so result is now")
print("----------------")
for i in range(len(txt_src)):
    print(txt_src[i].strip("\n"))

We could replace things:

In [ ]:
txt_src[-1].replace("Never", "Always")

> <span style="color:red">**Q.**</span> what is the `-1`, and why does this output still have the `\n` in it?

Writing to a file is similar: we open a pointer (this time with `w` for read), use the pointers `.write()` command to write a line, and when we are done, close it.

> NOTE: you need the `\n` to tell Python to write a new line.

Writing would generally be useful for example if there is some data you analysed, and re-running the code will take a while, or you want to share the processed results with someone.

In [ ]:
txt_file = open("rick_roll_troll.txt", "w")
for i in range(len(txt_src)):
    txt_file.write(txt_src[i].replace("Never", "Always"))
txt_file.close()

> <span style="color:red">**Q.**</span> what is the above code doing?

The code below provides an alternative way to opening a file that closes it immediate after reading, through the `with` command.

In [ ]:
with open("rick_roll_troll.txt", "r") as f:
    mod_txt = f.readlines()
for line in range(len(mod_txt)):
    print(mod_txt[line].replace("\n", ""))

--------------

# b) Reading numeric data but in text format

Lets try a slightly less stupid example but one that highlights some generic things to watch out for. The (text) file `elnino34_sst.data` contains sea surface temperature (SST) data over the El-Nino 3.4 region.

> NOTE: if you don't know what El-Nino is, go to Wikipedia, OCES 2003 notes, or ENVS 3004 / OCES 4001 notes

Lets just open it and see what it contains.

In [ ]:
with open("elnino34_sst.data", "r") as f:
    elnino34_txt = f.readlines()
elnino34_txt

So while I read the file as a list, the entries we are primarily interested in are numbers, so we want a way to pull the numbers out. Also, note that:

1) There are a few unnecessary line at the beginning (the **headers**) and at the end (the **footers**), that we should get rid of probably for the data analysis purposes.

2) The year is on the left column, but we probably don't need that.

3) It looks like there are 12 columns, which in this case really does correspond to entries to each of the months.

> NOTE: I think (but I can't remember) this is actually raw monthly averaged SST values, rather than averaged over a window to give *low-passed* data; see *07_time_series* for what this means if you want.

4) there are entries of `-99.99`, which in this case are **missing values** (no data), which you probably want to do something about

> NOTE: Sometimes the missing values show up as something else, or marked on differently, so you do need to check for these. How you deal with those is a separate issue (e.g. if the missing value occurs in the middle of some time, do you approximate it as something, fill it with something else, ignore it? The choice can affect the resulting analysis and is really guided by what question you are trying to answer.

Here I am going to process this the old-fashioned way to illustrate the points. I am going to

1) Strip out by hand the first 3 lines (header and the two lines with the missing values), and the last 4 lines (the footers).

2) Read each line, split out the entires somehow, but skip the first one (so not bothering with the year), noting that we are going from 1950 to 2019, and dump out the numbers per read.

3) Turn the eventual list into an array.

The code below does this, but for demonstration purposes, you should run the individual component separately (in different cells for example) to convince yourself what each part of the code is doing.

In [ ]:
with open("elnino34_sst.data", "r") as f:
    elnino34_txt = f.readlines()
elnino34_txt = elnino34_txt[3:-4]
for k in range(len(elnino34_txt)):
    elnino34_txt[k] = elnino34_txt[k].strip("\n")

elnino34_txt[0].split() # then we split each line (as a string) up into components

# define an empty list, cycle through each line, split, and add in the entries
elnino34_sst = []
for k in range(len(elnino34_txt)):           # this is the new elnino34_txt after stripping out some lines
    dummy = elnino34_txt[k].split()          # split out the entries per line
    for i in range(1, len(dummy)):           # cycle through the dummy list but skip the first entry
        elnino34_sst.append(float(dummy[i])) # turn string into a float, then add to list

elnino34_sst = np.asarray(elnino34_sst)      # turn into array (not strictly necessary)
print(elnino34_sst)

> <span style="color:red">**Q.**</span> What is the size of this array (look up how you might check), and is it exactly the same size as you expect it to be (what is the number of entries you should be getting)?

Do a raw plot of this to see what the data looks like.

In [ ]:
fig = plt.figure(figsize=(10, 3))
ax = plt.axes()
ax.plot(elnino34_sst)
ax.set_xlabel(r"index (not time)")
ax.set_ylabel(r"SST (${}^\circ\ \mathrm{C}$)")
ax.grid()

Notice here I haven't bothered to process the time (so there is no "time" to plot against), but it can in principle be generated. I could also have plotted straight from a list, but for data manipulation having an array is easier.

> <span style="color:red">**Q.**</span> (NEEDS CARE) Create a time vector manually so you can plot the above graph against a proper time showing a year (trying using `np.linspace`; look up syntax via `np.linspace?` or Google, being careful about the END year.

> <span style="color:red">**Q.**</span> Modify the code above so `elnino34_sst` stays a list, and try taking away a fixed number from it (say do `elnino34_sst - 2`), what happens? Do you get the same results from `elnino34_sst` as an array? Why?

## Introducing [pandas](https://pandas.pydata.org/)

<img src="https://i.imgur.com/rKcpZzr.jpg" width="400" alt='cursed panda'>

The Python Pandas package (not the cursed panda above; longer tutorial [here](https://www.w3schools.com/python/pandas/default.asp)) is a versatile package that is particularly good for packaging data and doing relevant analysis on.

Whatever I am going to be doing over for the rest of this course can probably be done through Pandas in principle, but I am mostly not going to be doing things through Pandas, as I think it hides a bit of the detail I think is important to go through somewhat. But an ongoing exercise would be for you do whatever I am doing but in Pandas.

Anyway, reading data can be done through the `read_csv` command that comes with pandas.

In [ ]:
import pandas as pd

data = pd.read_csv("elnino34_sst.data")
print(data)

So generally pandas ***tries*** to read things assuming sensible layout etc., but that can fail if the data is not cleaned up (and uncleaned data is the default rather than the norm). So, it is generally advised to have a look at the raw data file first to see what it consists, anticipate what things you might need to do, and  know back up options (hence somewhat of a reason for the above manipulations in the old-fashioned way).

> NOTE: Again, it is probably good to accept that data you will get your hands on is almost never cleaned up, learn what kind of problems can get thrown up and learn how to deal with it, instead of always getting clean data and not knowing what to do when data you get is no longer clean.

In this case, optional arguments needs to be provided (e.g. delimiter, separator, etc...). I've done the whole thing below.

> <span style="color:red">**Q.**</span> Experiment on what happens if you mess around with the options below and what outputs you get (giving you some experience as to what the "wrong" things look like)

In [ ]:
# can give it a few more details to make it easier for pandas to help us
data = pd.read_csv("elnino34_sst.data",
            delim_whitespace=True,
            names=["year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
            skipfooter=4,  # chop out some lines
            skiprows=1,    # chop out some unnecessary lines
            false_values=-99.99,
            engine="python")
data = data.replace(-99.99, np.nan) # replace missing values with NaNs (not a number)
data = data.set_index("year")       # sets the index to be the year column
data

You can think of the Pandas data frame like a dictionary, so you could for example do the following.

In [ ]:
fig = plt.figure(figsize=(10, 3))

# forces pandas to plot on the given ax object, otherwise it generates a new one
ax = plt.axes()
data["Jan"].plot(ax=ax) 

# because year is set as index, so data is being plotted against index
#  can modify this accordingly
ax.set_xlabel(r"year")
ax.set_ylabel(r"SST (${}^\circ\ \mathrm{C}$)")
ax.grid()

Note you do have a time-axis here, because the `year` column was not discarded in this case, but it is of course less obvious how you plot the whole time-series out.

> <span style="color:red">**Q.**</span> What is the above doing?

> <span style="color:red">**Q.**</span> With a for loop or otherwise, plot the data for all 12 months of data at the same time.

The code below is basically the same as above, but here I pull out the data in the Pandas data frame using the `.values` command (so now I am dealing with an array).

In [ ]:
# the code is does basically the same as above

year = data.index.values
sst  = data["Jan"].values

fig = plt.figure(figsize=(10, 3))
ax = plt.axes()
ax.plot(year, sst)
ax.set_xlabel(r"year")
ax.set_ylabel(r"SST (${}^\circ\ \mathrm{C}$)")
ax.grid()

> <span style="color:red">**Q.**</span> (harder) Try and see if you can bully Pandas into plotting out the whole data array as a complete time-series

--------------------

# c) Basic statistics (and some more Pandas)

Having a whole load of data is all well and good but we want some way to quantify these, so the rest of the course deals with that. The main kind of data we are going to be deal with are going to be **numbers**, and essentially we are going to be doing calculations for things.

For ease of talking, I am going to introduce some terminology and notation that I am not going to be very rigourous in defining. A **random variable** $X$ (capital letters) following some **probability distribution** can be sampled, and I am going to denote the **samples** of the random variable as $x_i$ (small letters, with index $i$ denoting the $i^{\rm{th}}$ sample).

> NOTE: $X$ really is a map from the sample space $\Omega$ (space of all possible outcomes) to some measurable space $E$, but we don't really need this here...

For most intents and purposes, we are going to be dealing with $x_i$, the data, and occasionally the probability distribution function or the random variable, but otherwise you can largely forget about the rest of the stuff I just said.

## Average

So there are really three kinds of *averages*, but usually when we talk about THE average, we really mean the **mean**. We are going to skip that for the moment and talk about the other two first. 

1) **Mode**, the sample value that occurs the most.

2) **Median**, order the samples, and pick the middle one.

We will usually not use the mode, but the median will show up when we do the box-and-whisker plots (as the middle line, or the **50th quartile**).

`Numpy` has a built in function for this.

In [ ]:
ages = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

print(f"median of ages = {np.median(ages)}")

## Range and quartiles

The **range** of the sample is simply the distance between the smallest and largest value, and is a rather crude measure of spread.

The **upper/lower quartiles** or the **75/25 quartile** is also a measure of spread, but for data in the center. The upper quartile is the location above which 25% of the ranked data lies (or location of where 75% of the data lies below), and the lower quarter is analogously defined.

These can be respectively done by a simple `max - min`, and for example the `np.percentile` command.

In [ ]:
print(f"range of ages = {np.max(ages)} - {np.min(ages)} = {np.max(ages) - np.min(ages)}")
print(f"lower quartile = {np.percentile(ages, 25)}")
print(f"mid   quartile = {np.percentile(ages, 50)}")
print(f"upper quartile = {np.percentile(ages, 75)}")
print("")
print(f"quartile of ages in one go = {np.percentile(ages, [25, 50, 75])}")

The range and quartiles can be shown in a diagram through a **box-and-whisker diagram** (or a **box plot**). The code below does a box plot, and the features are:

* The line in the box, which is the median.
* The edges of the box, which are the upper and lower quartiles, and the ranges is called the **inter-quartile range**.
* The whiskers, use to denote data outside of the inter-quartile range.
* The edges of the whiskers, which denote the range.

Sometimes when you use the built in boxplot commands extra circles appear. These usually denote the **outliers**, that are sufficiently far away from the median (usually measured by some factor of the inter-quartile range). You'll see this a bit more later when we look at the Iris data.

In [ ]:
# more plotting: box-and-whisker

fig = plt.figure(figsize=(5, 3))
ax = plt.axes()
ax.boxplot(ages)
ax.grid()
ax.set_ylabel(r"ages")

## Mean and variance / standard deviation

For $N$ samples with outcomes $x_i$, the **mean** of a sample $\overline{x}$ is defined as

\begin{equation*}
    \overline{x} = \frac{x_1 + x_2 + \ldots + x_N}{N} = \frac{1}{N}\sum_{i=0}^N x_i,
\end{equation*}

i.e. sum up all numbers (unweighted) and divide by the total number of samples.

> NOTE: I am going to be making a distinction in *05/06_statistical_tests* between the **sample mean** $\overline{x}$ and the **population mean** $\mu$ (the Greek character "mu", i.e. like the sound a cat makes).

So you need a way to count the number of samples, and to sum things. The code below shows how you might do this in a brute force way, a slightly more elegant way, and the easy way.

In [ ]:
# the really old fashioned way (partly to demonstrate some more syntax)
# compute the sum and count the number of numbers, then compute the mean
ages_avg, count = 0.0, 0
for i in range(len(ages)):
    count    += 1   # "count += 1" is equivalent to "count = count + 1"
    ages_avg += ages[i]

print(f"mean from the slow way = {ages_avg / count}")

# the cleaner way
ages_sum = np.sum(ages)
count = len(ages)
print(f"mean from sum / count  = {ages_sum / count}")

# the inbuilt way
print(f"mean from using np.mean= {np.mean(ages)}")

Most of the time, I am going use the unadjusted version of the **variance** $s^2$:

\begin{equation*}
    s^2 = \frac{(x_1 - \overline{x})^2 + (x_2 - \overline{x})^2 + \ldots + (x_N - \overline{x})^2}{N} = \frac{1}{N}\sum_{i=1}^{N} (x_i - \overline{x})^2,
\end{equation*}

i.e. take the sample mean from each sample away, square it, sum it, then divide by the number of samples. 

Two observations to makes here is that:

1) The variance is positive definite, and is only zero if every sample is the same as the sample mean.

2) If the sample size $N$ is large and only one sample is far from the mean, then that sample's contribution to the variance is rather minimal (because of the division by $N$).

From this, convince yourself that the variance is a measure of spread, and deviation away from the mean. The (unadjusted) **standard deviation**, which I will be short handing as s.t.d., partly for the puns I can make, is just the square root of the variance.

> NOTE: I am also going to make a distinction in *05/06_statistical_tests* between the **sample s.t.d.** $s$, and the **population s.t.d.** (heh) $\sigma$

The code below does the same as the calculations for the variance and the s.t.d., with one verbose way, one easier way, and one using the inbuilt command in `numpy` (`np.std`).

In [ ]:
# demonstrate with only mean and variance for now

ages = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
ages_avg = np.mean(ages)

# take the mean off every number in the array, and then compute things needed for standard deviation
# I am going to compute the variance then work out the s.d.
ages_var, count = 0.0, 0  # reset the counter
for i in range(len(ages)):
    count    += 1
    ages_var +=(ages[i] - ages_avg)**2

ages_var /= count

print(f"stats from verbose code: mean = {ages_avg}, variance = {ages_var}, s.t.d. = {np.sqrt(ages_var):.3f}")

# slightly less verbose way of doing above
ages_var = np.sum((ages - ages_avg)**2) / len(ages)
print(f"stats from cleaner code: mean = {ages_avg}, variance = {ages_var}, s.t.d. = {np.sqrt(ages_var):.3f}")

# using the inbuilt s.t.d.
print(f"stats from command     : mean = {np.mean(ages)}, variance = {np.var(ages)}, s.t.d. = {np.std(ages):.3f}")

### !!! USER BEWARE !!!

So here is an example why sometimes one should know the old-fashioned way of doing things, and be very careful when using inbuilt commands. Suppose I use instead another command for standard deviation from a different package (in this case the `statistics` package), and do the same thing above, I would get the following.

In [ ]:
import statistics as stat

print(f"""stats from another package: 

mean = {stat.mean(ages)}, variance = {stat.variance(ages)}, s.d. = {stat.stdev(ages):.3f}""")

The mean is the same but the variance and the s.t.d's (heh) are different, whats the deal?

I was quite explicit in saying I was using the unadjusted variance / s.t.d., and there is an adjusted version with what's called the **Bessel correction**,

\begin{equation*}
    s^2 = \frac{(x_1 - \overline{x})^2 + (x_2 - \overline{x})^2 + \ldots + (x_N - \overline{x})^2}{N - 1} = \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \overline{x})^2.
\end{equation*}

The variance and s.t.d. calculations in `numpy` uses the unadjusted definition, while the `statistics` packages uses the adjusted version. If you blindly use packages you would probably get different answers (like some of the students did in ENVS 3004 at some point, when they tried to make it easier for themselves and not do it by hand, and thus got it wrong because I explicitly asked them to use the definition I provided).

***Which should you use?*** I will be explicit in assignments. Personally I almost never deal with small samples or with statistical tests, so I tend to go with the unadjusted one (for large $N$, $N \approx N - 1$ so it makes very little practical difference). If you deal with small samples you probably should use the adjusted version. You can still use `numpy`, but override the default in this case with the keyword `ddof` (look this up in the manual if you want).

In [ ]:
print(f"""stats from numpy with override (ddof=0 by default): 

mean = {np.mean(ages)}, variance = {np.var(ages, ddof=1)}, s.d. = {np.std(ages, ddof=1):.3f}""")

----------------

# More involved exercises with this notebook

## 1) Iris data, stats and pandas

The [iris data](https://en.wikipedia.org/wiki/Iris_flower_data_set) was originally compiled by Ronald Fisher and is generally regarded as the *hello_world* of data analysis (and machine learning as well). We are going to be using that dataset quite a bit up to *06_statistical_tests* to demonstrate code syntax.

> NOTE: Since this is a well-known dataset, a lot of the answers you might want are actually readily found on Google.

> Random unrelated fact: cats are quite allergic to iris flowers (and quite a lot of flowers actually), so keep them away

I touched up the iris data from the [UCL repository](https://archive.ics.uci.edu/ml/datasets/iris) and dumped it out into a text file called `iris.csv`, which should be able to be read into Pandas directly. Try and explore the Iris data a bit and do some statistics/plots on it (I am not going to describe the data, you can look this up on the Wikipedia above). A few bits of possibly useful sample code are given below.

> <span style="color:red">**Q.**</span> Open `iris.csv` outside of python and see what it contains (you might need to chase up Wikipedia or the original UCL repository).

> <span style="color:red">**Q.**</span> Reproduce some of the statistics given by the `df.describe()` by hand, you might want some to use some of the codes below.

> <span style="color:red">**Q.**</span> Do some standard plots or box plots of various attributes, remember to label your graphs.

> <span style="color:red">**Q.**</span> Try and do a boxplot of say one attribute of the three different species of flower together (Google will probably give you the syntax).

> <span style="color:red">**Q.**</span> How would do you a scatter plot with `plt`? could you use `plt.plot` directly?

> <span style="color:red">**Q.**</span> (more involved) Provide some qualitative descriptions of the different attributes and its dependence on species. Note some of these down, as we will be quantifying these in the next four sessions.

In [ ]:
# pull files from the internet if needed (e.g. temporary session in Colab)
# !wget https://raw.githubusercontent.com/julianmak/OCES3301_data_analysis/main/iris.csv

df = pd.read_csv("iris.csv")
print("===============================")
print("sample of data")
print("===============================")
print(df.sample(10))
print()
print("===============================")
print("some basic statistics from data")
print("===============================")
print(df.describe())

# Q. it is probably also useful to have a look at the raw file itself to see what the data actually contains
#    open the file itself and see what units and variables the data contains

In [ ]:
# slice out one of the species and have a look at the basic statistics
setosa_data = df.loc[df["variety"] == "Setosa"]

# call a particular attribute
print(setosa_data["sepal.width"])